In [1]:
import s3fs
import polars as pl

In [2]:
fs = s3fs.S3FileSystem(anon=True)

In [47]:
files = fs.glob("noaa-gfs-bdp-pds/gdas.20210323/00/*")
len(files), files

(2,
 ['noaa-gfs-bdp-pds/gdas.20210323/00/atmos',
  'noaa-gfs-bdp-pds/gdas.20210323/00/wave'])

In [3]:
files = fs.glob("noaa-gfs-bdp-pds/gfs.20240401/*/atmos/*.idx")
len(files)

5356

In [40]:
files = fs.glob("noaa-gfs-bdp-pds/gfs.20240401/*/wave/gridded/*.idx")
len(files)

(5852,
 ['noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f000.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f001.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f002.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f003.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f004.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f005.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f006.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f007.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f008.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f009.grib2.idx',
  'noaa-gfs-bdp-pds/gfs.20240401/00/wave/gridded/gfswave.t00z.arctic.9km.f010.grib2.idx',
  '

In [43]:
files = fs.glob("noaa-gfs-bdp-pds/gdas.20240401/*/atmos/*.idx")
len(files)

100

In [44]:
df = (
    pl.DataFrame({"files": files})
    .filter(
        pl.col("files").str.ends_with(".idx"),
    )
    .with_columns(
        pl.col("files").str.split("/").list.last(),
        pl.col("files")
        .str.split("/")
        .list.last()
        .str.replace(r"f\d\d\d", "f###")
        .str.replace(r"t\d\dz", "tHHz")
        .alias("filename"),
    )
)
for i, f in enumerate(df["filename"].unique().sort(), start=1):
    print(f"{i:3}: {f}")

  1: gdas.tHHz.pgrb2.0p25.f###.idx
  2: gdas.tHHz.pgrb2.1p00.anl.idx
  3: gdas.tHHz.pgrb2.1p00.f###.idx
  4: gdas.tHHz.sfluxgrbf###.grib2.idx


In [9]:
df = (
    pl.DataFrame({"files": files})
    .filter(
        pl.col("files").str.ends_with(".idx"),
    )
    .with_columns(
        pl.col("files").str.split("/").list.last(),
        pl.col("files")
        .str.split("/")
        .list.last()
        .str.split(".")
        .list.to_struct()
        .alias("parts"),
    )
    .unnest("parts")
)
df

files,field_0,field_1,field_2,field_3,field_4,field_5
str,str,str,str,str,str,str
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f000""","""idx"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f003""","""idx"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f006""","""idx"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f009""","""idx"""
"""gfs.t00z.goessimpgrb2.0p25.f01…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f012""","""idx"""
…,…,…,…,…,…,…
"""gfs.t00z.sfluxgrbf372.grib2.id…","""gfs""","""t00z""","""sfluxgrbf372""","""grib2""","""idx""",null
"""gfs.t00z.sfluxgrbf375.grib2.id…","""gfs""","""t00z""","""sfluxgrbf375""","""grib2""","""idx""",null
"""gfs.t00z.sfluxgrbf378.grib2.id…","""gfs""","""t00z""","""sfluxgrbf378""","""grib2""","""idx""",null


In [39]:
df1 = df.filter(pl.col("field_4").is_null().not_())
df1

files,field_0,field_1,field_2,field_3,field_4
str,str,str,str,str,str
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f000"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f003"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f006"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f009"""
"""gfs.t00z.goessimpgrb2.0p25.f01…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f012"""
…,…,…,…,…,…
"""gfs.t00z.pgrb2full.0p50.f375""","""gfs""","""t00z""","""pgrb2full""","""0p50""","""f375"""
"""gfs.t00z.pgrb2full.0p50.f378""","""gfs""","""t00z""","""pgrb2full""","""0p50""","""f378"""
"""gfs.t00z.pgrb2full.0p50.f381""","""gfs""","""t00z""","""pgrb2full""","""0p50""","""f381"""


In [46]:
df1["field_0"].unique().to_list(), df1["field_1"].unique().to_list()

(['gfs'], ['t00z'])

In [47]:
df1["field_2"].unique()

field_2
str
"""goessimpgrb2"""
"""syndata"""
"""pgrb2full"""
"""pgrb2"""
"""pgrb2b"""


In [48]:
df1["field_3"].unique()

field_3
str
"""tcvitals"""
"""0p25"""
"""1p00"""
"""0p50"""


In [52]:
df1["field_4"].unique().sort()

field_4
str
"""anl"""
"""f000"""
"""f001"""
"""f002"""
"""f003"""
…
"""f375"""
"""f378"""
"""f381"""


In [53]:
df2 = df.filter(pl.col("field_4").is_null())
df2

files,field_0,field_1,field_2,field_3,field_4
str,str,str,str,str,str
"""gfs.t00z.goessimpgrb2f000.grd2…","""gfs""","""t00z""","""goessimpgrb2f000""","""grd221""",null
"""gfs.t00z.goessimpgrb2f003.grd2…","""gfs""","""t00z""","""goessimpgrb2f003""","""grd221""",null
"""gfs.t00z.goessimpgrb2f006.grd2…","""gfs""","""t00z""","""goessimpgrb2f006""","""grd221""",null
"""gfs.t00z.goessimpgrb2f009.grd2…","""gfs""","""t00z""","""goessimpgrb2f009""","""grd221""",null
"""gfs.t00z.goessimpgrb2f012.grd2…","""gfs""","""t00z""","""goessimpgrb2f012""","""grd221""",null
…,…,…,…,…,…
"""gfs.t00z.wgne.f168""","""gfs""","""t00z""","""wgne""","""f168""",null
"""gfs.t00z.wgne.f171""","""gfs""","""t00z""","""wgne""","""f171""",null
"""gfs.t00z.wgne.f174""","""gfs""","""t00z""","""wgne""","""f174""",null


In [55]:
df2["field_0"].unique().to_list(), df1["field_1"].unique().to_list()

(['gfs'], ['t00z'])

In [74]:
df2["field_2"].str.replace(r"f\d\d\d$", "").unique().sort()

field_2
str
"""goessimpgrb2"""
"""sfluxgrb"""
"""wgne"""


In [71]:
df2["field_3"].unique().sort()

field_3
str
"""f003"""
"""f006"""
"""f009"""
"""f012"""
"""f015"""
…
"""f174"""
"""f177"""
"""f180"""
